<a href="https://colab.research.google.com/github/Hira63S/DS-Unit-1-Sprint-2-Data-Wrangling/blob/master/Hira_LS_DS_122_Join_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science_

# Join datasets

Objectives
- concatenate data with pandas
- merge data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
  - Combine Data Sets: Standard Joins
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join

## Download data

We’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!

In [2]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-03-26 21:57:15--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.137.30
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.137.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  31.3MB/s    in 7.2s    

2019-03-26 21:57:23 (27.4 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [3]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [4]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [5]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


## Goal: Reproduce this example

The first two orders for user id 1:

In [6]:
from IPython.display import display, Image
url = 'https://cdn-images-1.medium.com/max/1600/1*vYGFQCafJtGBBX5mbl0xyw.png'
example = Image(url=url, width=600)

display(example)

## Load data

Here's a list of all six CSV filenames

In [7]:
!ls -lh

total 681M
-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


For each CSV
- Load it with pandas
- Look at the dataframe's shape
- Look at its head (first rows)
- `display(example)`
- Which columns does it have in common with the example we want to reproduce?

### aisles

In [0]:
import pandas as pd
aisles = pd.read_csv('aisles.csv')


In [9]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [10]:
aisles.shape

(134, 2)

### departments

In [0]:
departments = pd.read_csv('departments.csv')

In [12]:
departments.shape

(21, 2)

### order_products__prior

In [0]:
prior_orders = pd.read_csv('order_products__prior.csv')

In [14]:
prior_orders.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [15]:
prior_orders.head(10)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


In [16]:
prior_orders.groupby('order_id')['product_id'].count().mean()

10.088883421247614

### order_products__train

In [0]:
orders_train = pd.read_csv('order_products__train.csv')

In [18]:
orders_train.head(20)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
5,1,13176,6,0
6,1,47209,7,0
7,1,22035,8,1
8,36,39612,1,0
9,36,19660,2,1


### order-products_prios has 
-order_id
_product_id
-add_to_card_order
-reordered

### orders

In [0]:
orders = pd.read_csv('orders.csv')

In [20]:
orders.shape

(3421083, 7)

In [21]:
orders.head(3)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0


### products

In [0]:
products = pd.read_csv('products.csv')

In [23]:
products.head

<bound method NDFrame.head of        product_id                                       product_name  \
0               1                         Chocolate Sandwich Cookies   
1               2                                   All-Seasons Salt   
2               3               Robust Golden Unsweetened Oolong Tea   
3               4  Smart Ones Classic Favorites Mini Rigatoni Wit...   
4               5                          Green Chile Anytime Sauce   
5               6                                       Dry Nose Oil   
6               7                     Pure Coconut Water With Orange   
7               8                  Cut Russet Potatoes Steam N' Mash   
8               9                  Light Strawberry Blueberry Yogurt   
9              10     Sparkling Orange Juice & Prickly Pear Beverage   
10             11                                  Peach Mango Juice   
11             12                         Chocolate Fudge Layer Cake   
12             13                 

** prodcuts has:
- product_id
-product_name

## Concatenate order_products__prior and order_products__train

In [0]:
#both products prior and products train
order_products = pd.concat([prior_orders, orders_train])

In [25]:
order_products.shape, prior_orders.shape, orders_train.shape

((33819106, 4), (32434489, 4), (1384617, 4))

In [0]:
assert (len(order_products)) == len(prior_orders) + len(orders_train)

In [0]:
assert (len(order_products.columns)==
       len(prior_orders.columns) ==
        len(orders_train.columns))

In [0]:
rows, columns = order_products.shape

In [29]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


## Get a subset of orders — the first two orders for user id 1

**from orders dataframe:**
- user_id
- order_id
- order_number
- order_dow
- order_hour_of_day

In [30]:
condition = order_products['order_id'] == 2539329

order_products[condition]

,order_id,product_id,add_to_cart_order,reordered
24076664,2539329,196,1,0
24076665,2539329,14084,2,0
24076666,2539329,12427,3,0
24076667,2539329,26088,4,0
24076668,2539329,26405,5,0


In [31]:
orders['order_number'].describe()

count    3.421083e+06
mean     1.715486e+01
std      1.773316e+01
min      1.000000e+00
25%      5.000000e+00
50%      1.100000e+01
75%      2.300000e+01
max      1.000000e+02
Name: order_number, dtype: float64

In [32]:
orders[(orders['user_id']==1) & (orders['order_number']<=2)] #

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0


In [0]:
conditions = (orders['user_id'] == 1) & (orders['order_number'] <= 2)

columns = ['user_id',
          'order_id',
          'order_number',
          'order_dow',
          'order_hour_of_day']

subset = orders.loc[conditions, columns]

## Merge dataframes

merge the subset from order with columns from order products

then merge with columns from 'products'

In [34]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [0]:
merged = pd.merge(subset, order_products[['order_id', 'add_to_cart_order', 'product_id']])




In [36]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [0]:
final= pd.merge(merged, products[['product_id', 'product_name']])


In [38]:
merged.shape, products[['product_id', 'product_name']].shape, final.shape

((11, 7), (49688, 2), (11, 8))

In [39]:
final

,user_id,order_id,order_number,order_dow,order_hour_of_day,add_to_cart_order,product_id,product_name
0,1,2539329,1,2,8,1,196,Soda
1,1,2398795,2,3,7,1,196,Soda
2,1,2539329,1,2,8,2,14084,Organic Unsweetened Vanilla Almond Milk
3,1,2539329,1,2,8,3,12427,Original Beef Jerky
4,1,2398795,2,3,7,3,12427,Original Beef Jerky
5,1,2539329,1,2,8,4,26088,Aged White Cheddar Popcorn
6,1,2398795,2,3,7,5,26088,Aged White Cheddar Popcorn
7,1,2539329,1,2,8,5,26405,XL Pick-A-Size Paper Towel Rolls
8,1,2398795,2,3,7,2,10258,Pistachios
9,1,2398795,2,3,7,4,13176,Bag of Organic Bananas


**Which columns have them?**

products columns, and order_number, order_id
add_to_cart_orders

# Assignment

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

## Stretch challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.

I need product_id, add_to_cart_order, order_id

In [0]:
prior_orders = pd.read_csv('order_products__prior.csv')

prior_orders.head()

In [0]:
orders_train = pd.read_csv('order_products__train.csv')

orders_train.head()

In [0]:
products = pd.read_csv('products.csv')

products.head()

Literally, redoing the same thing as we did in lecture today. Concatenating the two dataframe together into order_products

In [0]:
order_products = pd.concat([prior_orders, orders_train])

order_products.head() #putting both prior orders and orders train datasets together

In [44]:
order_products.shape

(33819106, 4)

We need:
user_id
order_id
product_id
product_name

I initially thought I need user_id or order_id as well but it isn't necessary for this challenge exactly so I got rid of it. 

In [0]:
condition = order_products['order_id'] >= 38000 # I just ran this to practice what we did in class. I did want to explore if there is a way for us to put like a string name like =='banana'
#and it would pull up all the orders that had bananas in it...?

order_products[condition]

THen, Merge!!
I tried doing this, I guess I need to go over Ryan's explanation again, but from my understanding of the cheatsheet, this should've gone through but it kept giving me error, so I went ahead to use the value_count() function for the product_id column and then make a dataframe with these columns together. 

In [0]:
final_list = pd.merge(order_products['product_id'], products[['product_id', 'product_name']])

final_list.names() # I kept getting error while i try to run this. 

In [0]:
top_products = (order_products['product_id'].value_counts())

top_products = pd.DataFrame({'Number_of_products': top_products.values, 'product_id': top_products.index, })

Then we add the product_names column because as Ryan said it doesn't matter which one you do first

In [77]:
final_list = pd.merge(top_products, products[['product_id', 'product_name']],
                     on = 'product_id')

final_list.head(10)

,Number_of_products,product_id,product_name
0,491291,24852,Banana
1,394930,13176,Bag of Organic Bananas
2,275577,21137,Organic Strawberries
3,251705,21903,Organic Baby Spinach
4,220877,47209,Organic Hass Avocado
5,184224,47766,Organic Avocado
6,160792,47626,Large Lemon
7,149445,16797,Strawberries
8,146660,26209,Limes
9,142813,27845,Organic Whole Milk


In [66]:
condition = final_list['Number_of_products'] >= 300000 

final_list[condition]

,Number_of_products,product_id,product_name
0,491291,24852,Banana
1,394930,13176,Bag of Organic Bananas


Yayyyy! So this finally worked!! After crashing the dataset multiple times I got it to work. Now, I need to work on the stretch challenge later.

In [78]:
top_products = pd.merge(orders_train[['product_id', 'order_id']], products[['product_id','product_name']])

top_products.head(10)

,product_id,order_id,product_name
0,49302,1,Bulgarian Yogurt
1,49302,816049,Bulgarian Yogurt
2,49302,1242203,Bulgarian Yogurt
3,49302,1383349,Bulgarian Yogurt
4,49302,1787378,Bulgarian Yogurt
5,49302,2445303,Bulgarian Yogurt
6,49302,2853065,Bulgarian Yogurt
7,49302,3231517,Bulgarian Yogurt
8,11109,1,Organic 4% Milk Fat Whole Milk Cottage Cheese
9,11109,68474,Organic 4% Milk Fat Whole Milk Cottage Cheese
